In [1]:
import pandas as pd
#scrape the postal codes table from the following Wikipedia page and transform the data into a pandas dataframe 
postal_codes = pd.read_html('http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', flavor='bs4')
df=postal_codes[0]

#make the first row column labels
df.columns = df.iloc[0]
df=df[1:]

#drop cells with a borough that is Not assigned
df=df[df.Borough != 'Not assigned']

#combine rows with the same postal code, with the neighbourhoods separated by a comma
df=df.groupby(['Postcode','Borough']).agg(lambda x: ','.join(x))
df=df.reset_index()

#make neighbourhood name the same as borough name ff a cell has a borough but a Not assigned neighbourhood
x=df.loc[df.Neighbourhood=='Not assigned']
df.loc[x.index,'Neighbourhood']=df.loc[x.index,'Borough']
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [2]:
df.shape

(103, 3)

In [2]:
!wget -q -O 'coordinates.csv' http://cocl.us/Geospatial_data
df_coordinates = pd.read_csv('coordinates.csv')
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [3]:
df['Latitude']=df_coordinates.Latitude
df['Longitude']=df_coordinates.Longitude
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [4]:
#specifically explore the downtown Toronto borough by slicing the original dataframe and creating a new dataframe of downtown Toronto
df_downtown = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
df_downtown.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [5]:
import folium
# create map of downtown Toronto
map_downtown = folium.Map(location=[43.6543, -79.3860], zoom_start=12)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_downtown['Latitude'], df_downtown['Longitude'], df_downtown['Borough'], df_downtown['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

In [6]:
#utilizing the Foursquare API to explore the neighbourhoods and segment them
CLIENT_ID = '14T2OGYXAYSX1QFUKOGI5MDHQYOPYE5EKUA5UANZ3FZHA4T1'
CLIENT_SECRET = '4DRRLOE1OBYLBARPYTJMLHJQLMXUB4PQCQJUPXKOZ3UPUNIA'
VERSION = '20180605'

In [7]:
#explore the Central Bay Street neighbourhood
import requests
neighbourhood_latitude = df_downtown.loc[7, 'Latitude']
neighbourhood_longitude = df_downtown.loc[7, 'Longitude']
neighbourhood_name = df_downtown.loc[7, 'Neighbourhood']

#get the top 100 venues that are in Central Bay Street within a radius of 500 meters

LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c64a523f594df7dc3a40e3b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 81,
  'suggestedBounds': {'ne': {'lat': 43.6624524045, 'lng': -79.38117421839567},
   'sw': {'lat': 43.6534523955, 'lng': -79.39359098160432}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '537d4d6d498ec171ba22e7fe',
       'name': "Jimmy's Coffee",
       'location': {'address': '82 Gerrard Street W',
        'crossStreet': 'Gerrard & LaPlante',
        'lat': 43.65842123574496,
        'lng': -79.38561319551111,
        'label

In [9]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
#clean the json and structure it into a pandas dataframe
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Jimmy's Coffee,Coffee Shop,43.658421,-79.385613
1,Tim Hortons,Coffee Shop,43.658570,-79.385123
2,Hailed Coffee,Coffee Shop,43.658833,-79.383684
3,The Queen and Beaver Public House,Gastropub,43.657472,-79.383524
4,Mercatto,Italian Restaurant,43.660391,-79.387664


In [11]:
#create a function to repeat the same process to all the neighbourhoods in downtown Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
downtown_venues = getNearbyVenues(names=df_downtown['Neighbourhood'],
                                   latitudes=df_downtown['Latitude'],
                                   longitudes=df_downtown['Longitude']
                                  )

Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie


In [12]:
#analyze Each Neighbourhood

# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
downtown_onehot['Neighbourhood'] = downtown_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,"Cabbagetown,St. James Town",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category
downtown_grouped = downtown_onehot.groupby('Neighbourhood').mean().reset_index()
downtown_grouped

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.00,0.00,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.01,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,...,0.00,0.00,0.00,0.012346,0.000000,0.000000,0.012346,0.000000,0.00,0.012346
5,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.050000,0.000000,0.050000,0.010000,0.000000,0.01,0.000000
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
7,Church and Wellesley,0.011494,0.011494,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011494,...,0.00,0.00,0.00,0.000000,0.011494,0.011494,0.000000,0.011494,0.00,0.011494
8,"Commerce Court,Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.000000
9,"Design Exchange,Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.00,0.00,0.01,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.000000


In [14]:
#print each neighbourhood along with the top 5 most common venues
num_top_venues = 5

for hood in downtown_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3      Thai Restaurant  0.04
4  American Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2          Restaurant  0.05
3  Italian Restaurant  0.04
4      Farmers Market  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0  Airport Terminal  0.14
1    Airport Lounge  0.14
2   Airport Service  0.14
3     Boat or Ferry  0.07
4   Harbor / Marina  0.07


----Cabbagetown,St. James Town----
                venue  freq
0         Coffee Shop  0.10
1                Café  0.06
2          Restaurant  0.06
3         Pizza Place  0.04
4  Italian Restaurant  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1                Café  0.06
2  Italian Restauran

In [15]:
#function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [16]:
#create the new dataframe and display the top 10 venues for each neighbourhood
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = downtown_grouped['Neighbourhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,American Restaurant,Thai Restaurant,Steakhouse,Bakery,Asian Restaurant,Bar,Restaurant,Gym
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Italian Restaurant,Café,Bakery,Beer Bar,Seafood Restaurant,Steakhouse,Farmers Market
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Terminal,Airport Lounge,Harbor / Marina,Boutique,Sculpture Garden,Plane,Boat or Ferry,Airport Gate,Airport
3,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Café,Park,Market,Bakery,Italian Restaurant,Pizza Place,Pub,Japanese Restaurant
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint,Ice Cream Shop,Bubble Tea Shop,Spa,Thai Restaurant,Sandwich Place


In [17]:
#run k-means to cluster the neighbourhood into 5 clusters
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

downtown_grouped_clustering =downtown_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 2, 3, 3, 0, 4, 0, 0, 3], dtype=int32)

In [18]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood
downtown_merged = df_downtown

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
downtown_merged = downtown_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

downtown_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,3,Coffee Shop,Restaurant,Café,Park,Market,Bakery,Italian Restaurant,Pizza Place,Pub,Japanese Restaurant
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Burger Joint,Restaurant,Men's Store,Café,Mediterranean Restaurant,Fast Food Restaurant
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,3,Coffee Shop,Bakery,Pub,Café,Park,Mexican Restaurant,Breakfast Spot,Theater,Dessert Shop,Shoe Store
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Thai Restaurant,Restaurant,Plaza,Diner,Pizza Place


In [19]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[43.6543, -79.3860], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighbourhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

KeyError: 'Cluster Labels'

In [38]:
#examine each cluster
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Burger Joint,Restaurant,Mediterranean Restaurant,Café,Men's Store,Pub
4,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Thai Restaurant,Ramen Restaurant,Plaza,Diner,Pizza Place
5,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Hotel,Clothing Store,Gastropub,Breakfast Spot,Park,Cocktail Bar,Cosmetics Shop
6,Downtown Toronto,0,Coffee Shop,Restaurant,Cocktail Bar,Seafood Restaurant,Farmers Market,Beer Bar,Steakhouse,Cheese Shop,Café,Italian Restaurant
8,Downtown Toronto,0,Coffee Shop,Café,American Restaurant,Thai Restaurant,Steakhouse,Restaurant,Hotel,Gym,Bakery,Bar
11,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gastropub,Gym,Bakery,Deli / Bodega,Steakhouse
12,Downtown Toronto,0,Café,Restaurant,Bookstore,Coffee Shop,Bar,Bakery,Japanese Restaurant,Sushi Restaurant,Sandwich Place,Massage Studio
13,Downtown Toronto,0,Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Coffee Shop,Bakery,Mexican Restaurant,Chinese Restaurant,Grocery Store
15,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Beer Bar,Seafood Restaurant,Italian Restaurant,Pub,Hotel,Cocktail Bar,Farmers Market
16,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Bar,Steakhouse,Bakery,Deli / Bodega,Seafood Restaurant


In [39]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant


In [40]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,2,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Boutique


In [41]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,3,Coffee Shop,Restaurant,Park,Italian Restaurant,Pub,Bakery,Pizza Place,Café,Gastropub,Pet Store
3,Downtown Toronto,3,Coffee Shop,Bakery,Pub,Park,Café,Theater,Breakfast Spot,Mexican Restaurant,Bank,Italian Restaurant
7,Downtown Toronto,3,Coffee Shop,Café,Italian Restaurant,Burger Joint,Bar,Sandwich Place,Thai Restaurant,Middle Eastern Restaurant,Spa,Chinese Restaurant
9,Downtown Toronto,3,Coffee Shop,Aquarium,Hotel,Café,Pizza Place,Italian Restaurant,Brewery,Restaurant,Scenic Lookout,Bakery
10,Downtown Toronto,3,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Gym,Italian Restaurant,Deli / Bodega,Gastropub,Beer Bar


In [42]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,4,Grocery Store,Café,Park,Nightclub,Diner,Baby Store,Restaurant,Athletics & Sports,Italian Restaurant,Convenience Store
